In [1]:
!pwd

/mnt/data/Data-Document/Data_CatalogBankPaper/Tools


In [3]:
%cd /mnt/data/Data-Document/Data_CatalogBankPaper/McMasterCarr/McMasterCarr_DocumentLabeler


/mnt/data/Data-Document/Data_CatalogBankPaper/McMasterCarr/McMasterCarr_DocumentLabeler


In [27]:
text_file = 'Label.txt'
target_directory = '/mnt/data/Data-Document/Data_CatalogBankPaper/McMasterCarr/McMasterCarr_DocumentLabeler/train'

convert_files(text_file, target_directory)

In [26]:
import json
import os
import shutil

def convert_files(text_file, target_directory):
    # function to convert the json data to the desired format
    def convert_json_data(data):
        id = 0
        converted_data = []
        for entry in data:
            transcription = entry['transcription']
            points = entry['points']
            label = entry['key_cls']
            x1, y1 = points[0]
            x2, y2 = points[2]
            box = [x1, y1, x2, y1, x2, y2, x1, y2]
            converted_data.append([id] + box + [transcription, label])
            id += 1
        return converted_data

    # create the directories if they don't exist
    boxes_and_transcripts_dir = os.path.join(target_directory, 'boxes_and_transcripts')
    images_dir = os.path.join(target_directory, 'images')
    os.makedirs(boxes_and_transcripts_dir, exist_ok=True)
    os.makedirs(images_dir, exist_ok=True)
    
    # read the text file
    with open(text_file, 'r') as f:
        file_data = f.readlines()
    
    tsv_files = []  # list to store the generated .tsv file paths
    
    # loop over each line in the text file
    for line in file_data:
        line = line.strip()
        file_path, json_data = line.split('\t')
        base_file_name = os.path.splitext(os.path.basename(file_path))[0]
        tsv_file_path = os.path.join(boxes_and_transcripts_dir, base_file_name + '.tsv')
        image_file_path = os.path.join(images_dir, base_file_name + '.jpg')
        
        # read the json data
        data = json.loads(json_data)

        # convert the json data to the desired format
        converted_data = convert_json_data(data)

        # write the output .tsv file
        with open(tsv_file_path, 'w') as f:
            # f.write('id\tX1\tY1\tX2\tY2\tX3\tY3\tX4\tY4\ttranscription\tlabel\n')
            for entry in converted_data:
                f.write(','.join([str(e) for e in entry]) + '\n')
        
        tsv_files.append(tsv_file_path)
        
        # copy the image file
        shutil.copy(base_file_name+'.jpg', image_file_path)
    
    # write the train_file_name.csv file
    with open(os.path.join(target_directory, 'train_file_name.csv'), 'w') as f:
        # f.write('id\tdocument\ttsv_file\n')
        for i, tsv_file in enumerate(tsv_files):
            f.write(str(i) + ',' + 'document' + ',' + os.path.splitext(os.path.basename(tsv_file))[0] + '.jpg' + '\n')
